# <center>Marking Guide question one</center>

1.	As part of this analysis, you will need to complete the following tasks: Create DataFrames from Various Data Sources: CSV Files: Read the sales data from a CSV file into a Spark DataFrame. JSON Files: Read the customer data from a JSON file into a Spark DataFrame. Parquet Files: Read the product data from a Parquet file into a Spark DataFrame. (10 Marks)

In [1]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder \
    .appName("Data Analysis Project") \
    .getOrCreate()

# Load CSV file into DataFrame
sales_df = spark.read.csv("../dataset/sales_data.csv", header=True, inferSchema=True)

# Load JSON file into DataFrame
customers_df = spark.read.json("../dataset/customers_data.json")

# Load Parquet file into DataFrame
products_df = spark.read.parquet("../dataset/products_data.parquet")

# Show sample data
print("Sales DataFrame:")
sales_df.show()
print("Customers DataFrame:")
customers_df.show()
print("Products DataFrame:")
products_df.show()


Sales DataFrame:
+--------------+----------+-----------+------+
|Transaction_ID|      Date|Customer_ID|Amount|
+--------------+----------+-----------+------+
|             1|2024-01-01|         37|178.94|
|             2|2024-01-02|          1| 236.3|
|             3|2024-01-03|         67|213.11|
|             4|2024-01-04|         24| 16.49|
|             5|2024-01-05|         27|155.01|
|             6|2024-01-06|         40| 235.0|
|             7|2024-01-07|         63|211.44|
|             8|2024-01-08|         61|363.75|
|             9|2024-01-09|         36|198.55|
|            10|2024-01-10|         98|249.79|
|            11|2024-01-11|         93|220.51|
|            12|2024-01-12|         24|372.91|
|            13|2024-01-13|         24|432.86|
|            14|2024-01-14|          4|270.63|
|            15|2024-01-15|         67|396.82|
|            16|2024-01-16|         34|155.43|
|            17|2024-01-17|         57|275.27|
|            18|2024-01-18|         49|179.

2.	Spark SQL and SQL Operations: Create Temporary Views: Register each DataFrame as a temporary view so that it can be queried using SQL. Basic SQL Operations: Use SQL queries to retrieve specific data (e.g., find the total sales per customer, list customers in a specific location). Perform joins between the DataFrames to combine the sales, customer, and product data based on common keys (e.g., join sales data with customer data on Customer_ID).(10 Marks)

In [2]:
#1. Register DataFrames as Temporary Views
sales_df.createOrReplaceTempView("sales")
customers_df.createOrReplaceTempView("customers")
products_df.createOrReplaceTempView("products")


In [3]:
#2. Basic SQL Operations
# Total sales per customer
total_sales_per_customer = spark.sql("""
    SELECT Customer_ID, SUM(Amount) AS Total_Sales
    FROM sales
    GROUP BY Customer_ID
""")
total_sales_per_customer.show()

# Customers in a specific location (e.g., "New York")
customers_in_location = spark.sql("""
    SELECT * 
    FROM customers
    WHERE Location = 'New York'
""")
customers_in_location.show()

# Join sales and customer data
sales_customers_joined = spark.sql("""
    SELECT s.Transaction_ID, s.Date, s.Customer_ID, c.Name, s.Amount
    FROM sales s
    JOIN customers c
    ON s.Customer_ID = c.Customer_ID
""")
sales_customers_joined.show()


+-----------+------------------+
|Customer_ID|       Total_Sales|
+-----------+------------------+
|         31|            109.68|
|         85|2227.5800000000004|
|         65|           1637.09|
|         53|           1612.51|
|         78|           1693.21|
|         34|            1417.0|
|         81|           1693.54|
|         28|           1466.08|
|         76|           1188.26|
|         27|           2258.75|
|         26|            507.81|
|         44|            236.15|
|         12|            754.08|
|         91|            950.87|
|         22|           2463.47|
|         93|             776.5|
|         47|             811.2|
|          1|2003.0300000000002|
|         52| 521.1899999999999|
|         13|            937.59|
+-----------+------------------+
only showing top 20 rows

+---+-----------+------+--------+----+
|Age|Customer_ID|Gender|Location|Name|
+---+-----------+------+--------+----+
+---+-----------+------+--------+----+

+--------------+---------

3. Advanced SQL Operations for Data Analysis: Aggregation and Grouping: Use SQL queries to perform advanced aggregations such as: Finding the total sales for each product category. Calculating the average transaction amount for customers from different locations. Sorting and Limiting Results: Sort the results of your queries by specific columns and limit the number of records returned. Filtering: Filter the data based on conditions (e.g., select customers over the age of 30 or products that have less than 10 units in stock). 				(10 Marks)

In [4]:
#1. Aggregation and Grouping
# Total sales for each product category
total_sales_per_category = spark.sql("""
    SELECT p.Category, SUM(s.Amount) AS Total_Sales
    FROM sales s
    JOIN products p
    ON s.Transaction_ID = p.Product_ID
    GROUP BY p.Category
""")
total_sales_per_category.show()

# Average transaction amount for customers by location
avg_transaction_by_location = spark.sql("""
    SELECT c.Location, AVG(s.Amount) AS Avg_Transaction
    FROM sales s
    JOIN customers c
    ON s.Customer_ID = c.Customer_ID
    GROUP BY c.Location
""")
avg_transaction_by_location.show()


+-----------+------------------+
|   Category|       Total_Sales|
+-----------+------------------+
|    Grocery|          35546.11|
|Electronics|28986.640000000003|
|   Clothing|          33633.55|
|  Furniture| 35241.65000000001|
+-----------+------------------+

+--------+-----------------+
|Location|  Avg_Transaction|
+--------+-----------------+
|  City_A|278.1170700636942|
|  City_B|247.2494375000002|
|  City_C|274.2276502732242|
+--------+-----------------+



In [5]:
#2. Sorting and Filtering
# Products with less than 10 units in stock
low_stock_products = spark.sql("""
    SELECT * 
    FROM products
    WHERE Stock < 10
""")
low_stock_products.show()

# Sort total sales by category in descending order
sorted_sales = total_sales_per_category.orderBy("Total_Sales", ascending=False)
sorted_sales.show()

+----------+------------+-----------+-----+------+
|Product_ID|Product_Name|   Category|Stock| Price|
+----------+------------+-----------+-----+------+
|         3|   Product_3|  Furniture|    5| 61.01|
|        14|  Product_14|  Furniture|    3| 51.62|
|        15|  Product_15|  Furniture|    9|152.82|
|        53|  Product_53|    Grocery|    7| 17.17|
|        57|  Product_57|  Furniture|    2|  66.8|
|        69|  Product_69|    Grocery|    6|102.61|
|        77|  Product_77|    Grocery|    4|190.72|
|        78|  Product_78|    Grocery|    2| 69.12|
|        85|  Product_85|    Grocery|    7| 30.68|
|        97|  Product_97|    Grocery|    8| 37.81|
|       133| Product_133|   Clothing|    2| 74.65|
|       160| Product_160|  Furniture|    9| 89.88|
|       162| Product_162|    Grocery|    4| 72.28|
|       165| Product_165|  Furniture|    5| 86.65|
|       178| Product_178|Electronics|    9|104.92|
|       180| Product_180|    Grocery|    7| 78.09|
|       197| Product_197|  Furn

4.	 Report Insights and Conclusion: Based on the results of your SQL queries, provide insights into the data: Identify trends such as top-selling products, high-value customers, and popular locations. Propose business strategies based on these insights (e.g., increasing stock for high-demand products, targeting specific customer demographics).			(10 Marks)

In [1]:
# Save results to CSV
#total_sales_per_category.write.csv("total_sales_per_category.csv", header=True)
#avg_transaction_by_location.write.csv("avg_transaction_by_location.csv", header=True)